In [1]:
# Core
import numpy as np
import pandas as pd
from scipy.interpolate import CubicSpline, RectBivariateSpline

# Astronomy
import astropy
from astropy.units import au, minute

from typing import List, Optional

In [2]:
# Local imports
import kepler_sieve
from asteroid_data import load_ast_pos
from asteroid_spline import get_df_shape, make_spline_df, spline_ast_data
# from asteroid_spline import make_spline_df, spline_ast_vec, spline_ast_elt
from planets_interp import get_earth_pos
from asteroid_direction import calc_dir_ast2obs, c
from db_utils import sp2df, df2db
import matplotlib.pyplot as plt

In [3]:
# Inputs
n0 = 0
n1 = 100

In [4]:
# Load asteroid positions
ast_pos = load_ast_pos(n0=n0, n1=n1)

In [5]:
# Reorder columns to put AsteroidID first
cols = ['AsteroidID', 'TimeID'] + list(ast_pos.columns[2:])
ast_pos = ast_pos[cols]

# Rename columns
col_tbl = {
    'mjd': 'tAst',
    'qx': 'qAst_x',
    'qy': 'qAst_y',
    'qz': 'qAst_z',
}
ast_pos.rename(columns=col_tbl, inplace=True)

In [6]:
ast_pos

,AsteroidID,TimeID,tAst,qAst_x,qAst_y,qAst_z
0,1,69120000,48000.0,-1.155142,2.297488,0.282872
1,1,69125760,48004.0,-1.192813,2.275319,0.289164
2,1,69131520,48008.0,-1.230156,2.252526,0.295376
3,1,69137280,48012.0,-1.267161,2.229116,0.301507
4,1,69143040,48016.0,-1.303818,2.205093,0.307555
...,...,...,...,...,...,...
371344,99,90696960,62984.0,2.347168,2.122528,0.009658
371345,99,90702720,62988.0,2.325895,2.148838,0.018000
371346,99,90708480,62992.0,2.304276,2.174829,0.026340
371347,99,90714240,62996.0,2.282313,2.200496,0.034676


In [7]:
# Copy asteroid positions into DataFrame used for directions
df = ast_pos.copy()

In [8]:
# Column names used in calculations
cols_q_ast = ['qAst_x', 'qAst_y', 'qAst_z']
cols_q_obs = ['qObs_x', 'qObs_y', 'qObs_z']

# The asteroid_id
asteroid_id = df.AsteroidID.values

# The time the light leaves the asteroid and arrives at the observer
t_ast = df.tAst.values
t_obs = t_ast
# Save t_obs to DataFrame
df['tObs'] = t_obs

# Spline earth vectors and update qObs on DataFrame; this will not change
q_obs = get_earth_pos(ts=t_obs)
df[cols_q_obs] = q_obs

# Extract asteroid position vectors
q_ast = df[cols_q_ast].values

# Add light time column; Initial guess was 0 (tObs = tAst + 0)
df['LightTime'] = 0.0

In [9]:
# Build asteroid position spline
id_col = 'AsteroidID'
time_col = 'tAst'
spline_q_ast = make_spline_df(df=ast_pos, cols_spline=cols_q_ast, id_col=id_col, time_col=time_col)

In [15]:
# New asteroid positions are splined output
q_ast = spline_q_ast(t_ast, asteroid_id)

# Save to DataFrame
df[cols_q_ast] = q_ast

In [16]:
df

,AsteroidID,TimeID,tAst,qAst_x,qAst_y,qAst_z,tObs,qObs_x,qObs_y,qObs_z,LightTime
0,1,69120000,48000.0,-1.155142,2.297488,0.282872,48000.0,-0.880033,-0.483797,-0.000050,0.0
1,1,69125760,48004.0,-1.192813,2.275319,0.289164,48004.0,-0.845941,-0.543194,-0.000054,0.0
2,1,69131520,48008.0,-1.230156,2.252526,0.295376,48008.0,-0.807882,-0.600059,-0.000056,0.0
3,1,69137280,48012.0,-1.267161,2.229116,0.301507,48012.0,-0.766061,-0.654113,-0.000055,0.0
4,1,69143040,48016.0,-1.303818,2.205093,0.307555,48016.0,-0.720707,-0.705117,-0.000054,0.0
...,...,...,...,...,...,...,...,...,...,...,...
371344,99,90696960,62984.0,2.347168,2.122528,0.009658,62984.0,-0.804421,-0.605790,0.000125,0.0
371345,99,90702720,62988.0,2.325895,2.148838,0.018000,62988.0,-0.762203,-0.659471,0.000130,0.0
371346,99,90708480,62992.0,2.304276,2.174829,0.026340,62992.0,-0.716489,-0.710085,0.000132,0.0
371347,99,90714240,62996.0,2.282313,2.200496,0.034676,62996.0,-0.667499,-0.757430,0.000132,0.0


In [19]:
def light_time_iter(df: pd.DataFrame, t_ast: np.ndarray, q_ast: np.ndarray):
    """
    One iteration of refinement of light time calculation; modifies DataFrame in place.
    INPUTS:
        df:     DataFrame assembled in function calc_light_time.
        t_ast:  Array of times when photons leave asteroid.
        q_ast:  Position of asteroids at these times.
    OUTPUTS:
        None.  Modifies df in place.
    """
    # Column names
    cols_q_obs = ['qObs_x', 'qObs_y', 'qObs_z']

    # Calculate t_ast from t_obs and current estimate of light_time
    light_time = df['LightTime'].values
    t_ast = t_obs - light_time / 1440.0             # 1440 is number of minutes in one day
    # Save revised time light leaves asteroid to DataFrame
    df['tAst'] = t_ast

    # Calculate new asteroid positions at the revised times
    q_ast = spline_q_ast(t_ast, asteroid_id)
    df[cols_q_ast] = q_ast

    # Compute position difference and distance from asteroid to earth
    dq = q_ast - q_obs
    r = np.sqrt(np.sum(dq*dq, axis=1))

    # Compute light time and update it on DataFrame
    light_time = r / c
    df['LightTime'] = light_time

In [20]:
    # Experiments show that 4 iterations is sufficient to achieve full convergence
    # Error in light_time (number of minutes) around 5E-9 at this point
    for _ in range(4):
        light_time_iter(df=df, t_ast=t_ast, q_ast=q_ast)

    # Compute position difference and distance from asteroid to earth
    q_obs = df[cols_q_obs].values
    dq = q_ast - q_obs
    r = np.sqrt(np.sum(dq*dq, axis=1)).reshape((-1, 1))

    # Calculate the direction and save it to DataFrame
    u = dq / r
    cols_dir = ['ux', 'uy', 'uz']
    df[cols_dir] = u

In [21]:
df

,AsteroidID,TimeID,tAst,qAst_x,qAst_y,qAst_z,tObs,qObs_x,qObs_y,qObs_z,LightTime,ux,uy,uz
0,1,69120000,47999.983776,-1.155142,2.297488,0.282872,48000.0,-0.880033,-0.483797,-0.000050,23.362923,-0.097934,0.990084,0.100715
1,1,69125760,48003.983514,-1.192658,2.275412,0.289138,48004.0,-0.845941,-0.543194,-0.000054,23.740465,-0.121519,0.987404,0.101321
2,1,69131520,48007.983258,-1.230000,2.252623,0.295350,48008.0,-0.807882,-0.600059,-0.000056,24.108877,-0.145674,0.984069,0.101916
3,1,69137280,48011.983008,-1.267005,2.229217,0.301481,48012.0,-0.766061,-0.654113,-0.000055,24.468004,-0.170329,0.980041,0.102504
4,1,69143040,48015.982766,-1.303660,2.205197,0.307529,48016.0,-0.720707,-0.705117,-0.000054,24.817330,-0.195416,0.975287,0.103088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371344,99,90696960,62983.975925,2.347295,2.122368,0.009607,62984.0,-0.804421,-0.605790,0.000125,34.668216,0.756050,0.654510,0.002287
371345,99,90702720,62987.975892,2.326025,2.148681,0.017950,62988.0,-0.762203,-0.659471,0.000130,34.715004,0.739820,0.672791,0.004281
371346,99,90708480,62991.975875,2.304407,2.174673,0.026290,62992.0,-0.716489,-0.710085,0.000132,34.740041,0.723167,0.690645,0.006274
371347,99,90714240,62995.975873,2.282446,2.200342,0.034626,62996.0,-0.667499,-0.757430,0.000132,34.743486,0.706111,0.708053,0.008269


In [3]:
# Calculate the direction and light time
df = calc_dir_ast2obs(n0=0, n1=100)

In [4]:
df

,AsteroidID,TimeID,tAst,qAst_x,qAst_y,qAst_z,vAst_x,vAst_y,vAst_z,LightTime,tObs,qObs_x,qObs_y,qObs_z,ux,uy,uz
0,1,69120000,48000.0,-1.155142,2.297488,0.282872,-0.009457,-0.005464,0.001583,23.365052,48000.016226,-0.879903,-0.484043,-0.000050,-0.097971,0.990081,0.100705
99,1,69125760,48004.0,-1.192813,2.275319,0.289164,-0.009377,-0.005620,0.001563,23.742002,48004.016488,-0.845792,-0.543434,-0.000054,-0.121560,0.987400,0.101312
198,1,69131520,48008.0,-1.230156,2.252526,0.295376,-0.009294,-0.005776,0.001543,24.110400,48008.016743,-0.807715,-0.600292,-0.000056,-0.145719,0.984064,0.101907
297,1,69137280,48012.0,-1.267161,2.229116,0.301507,-0.009208,-0.005929,0.001522,24.469509,48012.016993,-0.765876,-0.654337,-0.000055,-0.170378,0.980034,0.102496
396,1,69143040,48016.0,-1.303818,2.205093,0.307555,-0.009120,-0.006082,0.001501,24.818813,48016.017235,-0.720504,-0.705329,-0.000054,-0.195467,0.975278,0.103079
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370952,99,90696960,62984.0,2.347168,2.122528,0.009658,-0.005274,0.006617,0.002086,34.668563,62984.024075,-0.804178,-0.606122,0.000125,0.755986,0.654584,0.002287
371051,99,90702720,62988.0,2.325895,2.148838,0.018000,-0.005362,0.006538,0.002085,34.715220,62988.024108,-0.761938,-0.659785,0.000130,0.739754,0.672864,0.004281
371150,99,90708480,62992.0,2.304276,2.174829,0.026340,-0.005448,0.006457,0.002085,34.740126,62992.024125,-0.716203,-0.710380,0.000132,0.723099,0.690716,0.006274
371249,99,90714240,62996.0,2.282313,2.200496,0.034676,-0.005533,0.006376,0.002083,34.743441,62996.024127,-0.667194,-0.757706,0.000132,0.706041,0.708123,0.008269


In [5]:
# Rename column tAst back to mjd to match DB schema
df.rename(columns={'tAst':'mjd'}, inplace=True)

# Arguments to df2db
schema = 'KS'
table = 'AsteroidDirections2'
columns = ['AsteroidID', 'TimeID', 'mjd', 'ux', 'uy', 'uz', 'LightTime']
chunksize = 2**19
verbose = False
progbar = True

In [6]:
df2db(df=df, schema=schema, table=table, columns=columns, chunksize=chunksize, verbose=verbose, progbar=progbar)

  0%|          | 0/1 [00:00<?, ?it/s]